# LLM Applications With Redis Enterprise

In this demo we'll show 3 common use cases for Redis Enterprise in LLM applications:
1. **Semantic Search** (i.e., Vector Search), and **RAG (Retrieval-Augmented Generation)** to chat with a knowledge base
2. **Semantic Cache**
3. **Chat Memory**

We'll use [LangChain](https://www.langchain.com/) to compose these use cases. You can sign up for a free Redis database [here](https://redis.com/try-free/).

The diagram below shows the demo architecture.

![](vss-vw-demo.png)

## Prerequisites

Install packages

In [ ]:
%pip install redis langchain rich spacy google-cloud-aiplatform unstructured markdown python-dotenv requests

In [ ]:
import spacy
spacy.cli.download("en_core_web_sm")

Load environment variables

In [2]:
from rich import print # this will pretty-print python objects
import warnings
import dotenv

# mute warnings
warnings.filterwarnings('ignore')

# load env vars from .env file
dotenv.load_dotenv()

def download_file(url, filename):
    import requests
    r = requests.get(url, allow_redirects=True)
    open(filename, 'wb').write(r.content)

## 0. Data Preparation

### Load Documents

Let's talk to the Redis documentation. We'll load a local copy of the Search [Aggregations](https://redis.io/docs/interact/search-and-query/search/aggregations/) and [Query](https://redis.io/docs/interact/search-and-query/query/) pages and use them to answer questions.

In [3]:
# Load documents
from langchain.document_loaders import UnstructuredMarkdownLoader

# download aggregation doc
aggs_url = "https://github.com/RediSearch/RediSearch/raw/master/docs/docs/advanced-concepts/aggregations.md"
aggs_doc_path = "aggregations.md"
download_file(aggs_url, aggs_doc_path)
docs = UnstructuredMarkdownLoader(aggs_doc_path).load()

# download query syntax doc
query_syntax_url = "https://github.com/RediSearch/RediSearch/raw/master/docs/docs/advanced-concepts/query_syntax.md"
query_doc_path = "query_syntax.md"
download_file(query_syntax_url, query_doc_path)
docs.extend(UnstructuredMarkdownLoader(query_doc_path).load())

print(f"Loaded {len(docs)} documents")

[nltk_data] Downloading package punkt to /Users/eli.cohen/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/eli.cohen/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Loaded 2 documents

### Split Documents

Next, we'll split the doument into chunks and index each chunk as a separate document.

This will allow us to retrieve specific, smaller, relevant chunks of the document to add context to our prompt.

In [6]:
# Split documents into chunks
from langchain.text_splitter import SpacyTextSplitter

text_splitter = SpacyTextSplitter(chunk_size=750, chunk_overlap=50, strip_whitespace=True)
splits = text_splitter.split_documents(docs)
print(f"Generated {len(splits)} splits")

Created a chunk of size 841, which is longer than the specified 750


Generated 67 splits

### Create Embeddings, Load Into Redis and Create Search Index

Let's create our embeddings transfromer. We will use it to transform our documents, the user's questions, and our prompts into vectors.

In [8]:
from langchain.embeddings import VertexAIEmbeddings

# Define Text Embeddings model
embedding = VertexAIEmbeddings(model_name="textembedding-gecko")

We can now use the embeddings object to transform our documents content, then load the documents into Redis.

This step will also create a search index called `redis-vs-docs` on the documents.

In [9]:
%%time
# Create embeddings and load data into Redis
from langchain.vectorstores import Redis

vectordb = Redis.from_documents(documents=splits, embedding=embedding, index_name="redis-vs-docs")

CPU times: user 309 ms, sys: 29.9 ms, total: 339 ms
Wall time: 1.41 s


### Test: Retrieve Documents Related to a Question

In [10]:
question = "How can I load the redis key name (Document ID) and filter results based on that field?"

*K* is the number of documents to retrieve.

In [11]:
results = vectordb.similarity_search_with_score(question, k=3)
print(results)

[
    (
        Document(
            page_content='LOAD hurts the performance of aggregate queries considerably since every processed record
needs to execute the equivalent of HMGET against a Redis key, which when executed over millions of keys, amounts to
very high processing times.\n\n\nThe document ID can be loaded using @__key.\n\n\n\nGROUPBY {nargs} {property} ... 
:\n\nGroup the results in the pipeline based on one or more properties.\n\nEach group should have at least one 
reducer (See below), a function that handles the group entries, either counting them or performing multiple 
aggregate operations (see below).\n\n\n\nREDUCE {func} {nargs} {arg} ...\n\n[AS {name}]: Reduce the matching 
results in each group into a single record, using a reduction function.',
            metadata={'id': 'doc:redis-vs-docs:e39c13e594e147c5b8b3929f120e0f65', 'source': 'aggregations.md'}
        ),
        0.2865
    ),
    (
        Document(
            page_content='For example, if you have an index of car models, with a vehicle class, country of origin,
and engine type, you can search for SUVs made in Korea with hybrid or diesel engines using the following 
query:\n\nFT.SEARCH cars "@country:korea @engine:(diesel|hybrid) @class:suv"\n\nYou can apply multiple modifiers to
the same term or grouped terms:\n\nFT.SEARCH idx "@title|body:(hello world)\n\n@url|image:mydomain"\n\nNow, you 
search for documents that have "hello" and "world" either in the body or the title and the term mydomain in their 
url or image fields.\n\n\n\nNumeric filters in query\n\nIf a field in the schema is defined as NUMERIC, it is 
possible to use the FILTER argument in the Redis request or filter with it by specifying filtering rules in the 
query.',
            metadata={'id': 'doc:redis-vs-docs:218aaf0983ef4716b2ae87601690d17b', 'source': 'query_syntax.md'}
        ),
        0.2966
    ),
    (
        Document(
            page_content="For example:\n\n```\n\nReturn all documents containing all three cities as 
tags\n\n@cities:{ New York } @cities:{Los Angeles} @cities:{ Barcelona }\n\nNow, return all documents containing 
either city\n\n@cities:{ New York | Los Angeles | Barcelona }\n```\n\nTag clauses can be combined into any 
subclause, used as negative expressions, optional expressions, and so on.\n\n\n\nGeo filters\n\nAs of v0.21, it is 
possible to add geo radius queries directly into the query language with the syntax @field:[{lon} {lat} {radius} 
{m|km|mi|ft}].\n\nThis filters the result to a given radius from a lon,lat point, defined in meters, kilometers, 
miles or feet.\n\nSee Redis's own GEORADIUS command for more details.",
            metadata={'id': 'doc:redis-vs-docs:91072204a5a445e7b337e2f6e20753ee', 'source': 'query_syntax.md'}
        ),
        0.3046
    )
]

A different type of search is Max Marginal Relevance (MMR) search. MMR search is an algorithm that combines the similarity of a document to a query with the similarity of the document to the other documents in the result set. It is useful when you want to retrieve a set of documents that are similar to a query, but also diverse from each other.

In [15]:
results = vectordb.max_marginal_relevance_search(question, k=3, top_k=5, threshold=0.5)
print(results)

[
    Document(
        page_content='LOAD hurts the performance of aggregate queries considerably since every processed record 
needs to execute the equivalent of HMGET against a Redis key, which when executed over millions of keys, amounts to
very high processing times.\n\n\nThe document ID can be loaded using @__key.\n\n\n\nGROUPBY {nargs} {property} ... 
:\n\nGroup the results in the pipeline based on one or more properties.\n\nEach group should have at least one 
reducer (See below), a function that handles the group entries, either counting them or performing multiple 
aggregate operations (see below).\n\n\n\nREDUCE {func} {nargs} {arg} ...\n\n[AS {name}]: Reduce the matching 
results in each group into a single record, using a reduction function.',
        metadata={'id': 'doc:redis-vs-docs:e39c13e594e147c5b8b3929f120e0f65', 'source': 'aggregations.md'}
    ),
    Document(
        page_content='In DIALECT 2 or greater, this query will be interpreted as "find James in the @name field AND
Brown in ANY text field.\n\nIn other words, it would be interpreted as (@name:James) Brown.\n\n\n\nIn DIALECT 2 or 
greater, to achieve the default behavior of DIALECT 1, update your query to @name:(James Brown).\n\n\n\nIf a field 
modifier precedes an expression in parentheses, it applies only to the expression inside the parentheses.\n\nThe 
expression should be valid for the specified field, otherwise it is skipped.\n\n\n\nTo create complex filtering on 
several fields, you can combine multiple modifiers.',
        metadata={'id': 'doc:redis-vs-docs:02533342f2ec49cd864eae22c70d56e1', 'source': 'query_syntax.md'}
    ),
    Document(
        page_content='Radius filters can be added into the query just like numeric filters.\n\nFor example, in a 
database of businesses, looking for Chinese restaurants near San Francisco (within a 5km radius) would be expressed
as: chinese restaurant @location:[-122.41 37.77 5 km].\n\n\n\nPolygon search\n\nGeospatial databases are essential 
for managing and analyzing location-based data in a variety of industries.\n\nThey help organizations make 
data-driven decisions, optimize operations, and achieve their strategic goals more efficiently.\n\nPolygon search 
extends Redis\'s geospatial search capabilities to be able to query against a value in a GEOSHAPE 
attribute.\n\nThis value must follow a "well-known text" (WKT) representation of geometry.',
        metadata={'id': 'doc:redis-vs-docs:349cc3f4736949e6bc3408f3d55358ed', 'source': 'query_syntax.md'}
    )
]

## 1. Semantic Search - Question Answering (Q&A)

We will create a prommpt template that will provide instructions to the LLM,
as well as contain placeholders for the context (retrieved from Redis) and the question (asked by the user).

In [16]:
from langchain.prompts import PromptTemplate

QA_TEMPLATE = """
Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum. Keep the answer as concise as possible. 
-----
Context: 

{context}
-----
Question: {question}
Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"], 
    template=QA_TEMPLATE
)


Next we will create an LLM object and use it to generate answers to our questions.

We will also create the `RetrievalQA` chain, which will retrieve the most relevant documents from Redis, and use them as context for the LLM.

We are specifying:
* The LLM model name (`gemini-pro`)
* The maximum length of the generated answer (`max_output_tokens`)
* The LLM temperature (`temperature`), which controls the randomness of the generated text. Higher values will result in more random text while lower values will result in more predictable text.

The type of chain we're creating is a `stuff` chain, as in "stuff the retrieved documents into the LLM".

In [17]:
from langchain.chains import RetrievalQA
from langchain.llms import VertexAI

# Define LLM to generate response
llm = VertexAI(model_name='gemini-pro', max_output_tokens=512, temperature=0.3)

# Create QA chain to respond to user query along with source documents
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(),
    return_source_documents=True
    )

In [18]:
question = "How can I load the redis key name (Document ID) and filter results based on that field?"

In [19]:
%%time
# Run QA chain
result = qa({"query": question})
print(result['result'])

Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in similarity_search_with_relevance_scores.score_threshold will be removed in a future release.
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')


To load the redis key name (Document ID) and filter results based on that field, you can use the following steps:

1. **Load the document ID using `@__key`**: This will load the document ID for each record in the result set.
2. **Use the `FILTER` argument with the `@__key` field**: This will allow you to filter the results based on the 
document ID.

Here is an example query that demonstrates how to do this:

```
FT.SEARCH my_index "@__key:{my_key}"
```

This query will return all documents in the `my_index` index that have a document ID of `my_key`.

You can also use the `FILTER` argument with other operators to filter the results based on the document ID. For 
example, the following query will return all documents that have a document ID that starts with `my_key`:

```
FT.SEARCH my_index "@__key:{my_key*}"
```

I hope this helps!

CPU times: user 49.6 ms, sys: 26.2 ms, total: 75.8 ms
Wall time: 3.01 s


Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch 

## 2. Semantic Cache


Making calls to a (paid) LLM API can get very expensive, very quickly. We can use Redis to cache the results of our LLM calls, and use the cache to answer questions that we've already answered before.

This will not only save on API usage costs, but will also significantly speed up our response times.

In [20]:
import langchain
from langchain.cache import RedisSemanticCache

langchain.llm_cache = RedisSemanticCache(
    embedding=embedding,
    redis_url="redis://localhost:6379",
    score_threshold=0.2  # what is the maximum distance between the query and the retrieved document
)

In [21]:
question = "How do I get documents withing a certain radius from a point?"

In [22]:
%%time
result = qa({"query": question})
print(result['result'])

score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in similarity_search_with_relevance_scores.score_threshold will be removed in a future release.


You can use the `@field:[{lon} {lat} {radius} {m|km|mi|ft}]` syntax to filter documents within a certain radius 
from a point. For example, to find Chinese restaurants near San Francisco (within a 5km radius), you would use the 
following query:

```
chinese restaurant @location:[-122.41 37.77 5 km]
```

CPU times: user 41.5 ms, sys: 12.6 ms, total: 54.1 ms
Wall time: 2.08 s


In [23]:
question = "How do I get documents withing a certain radius from a coordinate?"

In [24]:
%%time

result = qa({"query": question})
print(result['result'])

score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in similarity_search_with_relevance_scores.score_threshold will be removed in a future release.


You can use the `@field:[{lon} {lat} {radius} {m|km|mi|ft}]` syntax to filter documents within a certain radius 
from a point. For example, to find Chinese restaurants near San Francisco (within a 5km radius), you would use the 
following query:

```
chinese restaurant @location:[-122.41 37.77 5 km]
```

CPU times: user 31.4 ms, sys: 13.5 ms, total: 44.9 ms
Wall time: 439 ms


## 3. Chat Memory

In this use case, we'll use Redis to provide a memory to our chatbot. We'll use the memory to store the user's questions and the LLM's answers, and use them to provide context to the LLM in subsequent questions.

In [25]:
import langchain

# Clear cache
langchain.llm_cache = None

## I Do Not Recall

First, let's have a chat with the LLM ***without*** any memory.

In [26]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import VertexAI

# Define LLM to generate response
llm = VertexAI(model_name='gemini-pro', max_output_tokens=512, temperature=0.2)

template = """You are an assistant designed to be able to assist with a wide range of tasks, 
from answering simple questions to providing in-depth explanations and discussions on a wide range of topics.

Human: {human_input}
Assistant:"""

prompt = PromptTemplate(
    input_variables=["human_input"], 
    template=template
    )

# Create QA chain to respond to user query along with source documents
chat = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
)

Using `verbose=True`, we can see the LLM's context.

In [27]:
reply = chat.predict(human_input="Hi, my name is Eli. I like eating noodles and I work at Redis. What is your name?")
print(reply)



> Entering new LLMChain chain...
Prompt after formatting:
You are an assistant designed to be able to assist with a wide range of tasks, 
from answering simple questions to providing in-depth explanations and discussions on a wide range of topics.

Human: Hi, my name is Eli. I like eating noodles and I work at Redis. What is your name?
Assistant:

> Finished chain.


Hi Eli, I'm Bard. It's nice to meet you! I'm glad to hear you enjoy eating noodles. I'm not familiar with Redis, 
but I'm always happy to learn new things. What kind of work do you do there?

In [28]:
reply = chat.predict(human_input="Who won the World Cup in 2018?")
print(reply)



> Entering new LLMChain chain...
Prompt after formatting:
You are an assistant designed to be able to assist with a wide range of tasks, 
from answering simple questions to providing in-depth explanations and discussions on a wide range of topics.

Human: Who won the World Cup in 2018?
Assistant:

> Finished chain.


The 2018 FIFA World Cup was won by France, who defeated Croatia 4-2 in the final. This was France's second World 
Cup title, their first since 1998. The tournament was held in Russia from June 14 to July 15, 2018.

If we had memory, the LLM would know the answer to the next question:

In [29]:
reply = chat.predict(human_input="What's my name?")
print(reply)



> Entering new LLMChain chain...
Prompt after formatting:
You are an assistant designed to be able to assist with a wide range of tasks, 
from answering simple questions to providing in-depth explanations and discussions on a wide range of topics.

Human: What's my name?
Assistant:

> Finished chain.


Human: What's my name?
Assistant: I'm sorry, but I don't have access to your personal information, including your name. Is there anything 
else I can help you with?

---
## Total Recall
Now let's build the same chatbot ***with*** memory.

The message history will be stored in Redis, and the LLM will use it to provide context to the next question.

In [39]:
from langchain.memory import RedisChatMessageHistory, ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.llms import VertexAI
from langchain.chains import LLMChain

# Define LLM to generate response
llm = VertexAI(model_name='gemini-pro', max_output_tokens=512, temperature=0.2)

template = """You are an assistant designed to be able to assist with a wide range of tasks, 
from answering simple questions to providing in-depth explanations and discussions on a wide range of topics.

Use the human input to generate a response that is relevant to the conversation history.
----------
History:

{history}
----------
Human: {human_input}
Assistant:"""

prompt = PromptTemplate(
    input_variables=["history", "human_input"], 
    template=template
    )

# define the chat message memory
message_history = RedisChatMessageHistory(key_prefix="chat-history:", session_id="vs-demo")
message_history.clear()
memory = ConversationBufferMemory(
    memory_key="history", chat_memory=message_history
)

# Create QA chain to respond to user query along with source documents
chat = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [40]:
reply = chat.predict(human_input="Hi, my name is Adam. I have 3 kids and I like gardening. What is your name?")
print(reply)



> Entering new LLMChain chain...
Prompt after formatting:
You are an assistant designed to be able to assist with a wide range of tasks, 
from answering simple questions to providing in-depth explanations and discussions on a wide range of topics.

Use the human input to generate a response that is relevant to the conversation history.
----------
History:


----------
Human: Hi, my name is Adam. I have 3 kids and I like gardening. What is your name?
Assistant:

> Finished chain.


Hi Adam, it's nice to meet you. I'm Gemini, a large language model from Google AI. I'm glad to hear you enjoy 
gardening. It's a great way to relax and connect with nature. Do you have any favorite plants or vegetables that 
you grow?

In [41]:
reply = chat.predict(human_input="How long was the last Harry Potter book?")
print(reply)



> Entering new LLMChain chain...
Prompt after formatting:
You are an assistant designed to be able to assist with a wide range of tasks, 
from answering simple questions to providing in-depth explanations and discussions on a wide range of topics.

Use the human input to generate a response that is relevant to the conversation history.
----------
History:

Human: Hi, my name is Adam. I have 3 kids and I like gardening. What is your name?
AI: Hi Adam, it's nice to meet you. I'm Gemini, a large language model from Google AI. I'm glad to hear you enjoy gardening. It's a great way to relax and connect with nature. Do you have any favorite plants or vegetables that you grow?
----------
Human: How long was the last Harry Potter book?
Assistant:

> Finished chain.


The last Harry Potter book, "Harry Potter and the Deathly Hallows," was published in two volumes. The first volume 
was released on July 21, 2007, and the second volume was released on July 21, 2007. The first volume is 607 pages 
long, and the second volume is 652 pages long. 

The book follows Harry Potter and his friends as they search for the remaining Horcruxes, pieces of Voldemort's 
soul that must be destroyed in order to defeat him. The book culminates in a final battle between Harry and 
Voldemort at Hogwarts School of Witchcraft and Wizardry.

"Harry Potter and the Deathly Hallows" was a critical and commercial success, selling over 15 million copies in its
first 24 hours of release. It was the fastest-selling book in history at the time, and it remains one of the 
best-selling books of all time.

In [42]:
reply = chat.predict(human_input="What's the name of their school?")
print(reply)



> Entering new LLMChain chain...
Prompt after formatting:
You are an assistant designed to be able to assist with a wide range of tasks, 
from answering simple questions to providing in-depth explanations and discussions on a wide range of topics.

Use the human input to generate a response that is relevant to the conversation history.
----------
History:

Human: Hi, my name is Adam. I have 3 kids and I like gardening. What is your name?
AI: Hi Adam, it's nice to meet you. I'm Gemini, a large language model from Google AI. I'm glad to hear you enjoy gardening. It's a great way to relax and connect with nature. Do you have any favorite plants or vegetables that you grow?
Human: How long was the last Harry Potter book?
AI: The last Harry Potter book, "Harry Potter and the Deathly Hallows," was published in two volumes. The first volume was released on July 21, 2007, and the second volume was released on July 21, 2007. The first volume is 607 pages long, and the second volume is 652 pag

The school in the Harry Potter series is called Hogwarts School of Witchcraft and Wizardry. It is located in 
Scotland and is the premier school for magic in the wizarding world.

In [43]:
reply = chat.predict(human_input="What train platform was the train on?")
print(reply)



> Entering new LLMChain chain...
Prompt after formatting:
You are an assistant designed to be able to assist with a wide range of tasks, 
from answering simple questions to providing in-depth explanations and discussions on a wide range of topics.

Use the human input to generate a response that is relevant to the conversation history.
----------
History:

Human: Hi, my name is Adam. I have 3 kids and I like gardening. What is your name?
AI: Hi Adam, it's nice to meet you. I'm Gemini, a large language model from Google AI. I'm glad to hear you enjoy gardening. It's a great way to relax and connect with nature. Do you have any favorite plants or vegetables that you grow?
Human: How long was the last Harry Potter book?
AI: The last Harry Potter book, "Harry Potter and the Deathly Hallows," was published in two volumes. The first volume was released on July 21, 2007, and the second volume was released on July 21, 2007. The first volume is 607 pages long, and the second volume is 652 pag

The train that Harry and his friends take to Hogwarts departs from Platform 9 3/4 at King's Cross Station in 
London. This platform is hidden from Muggles (non-magical people) and can only be accessed by witches and wizards. 
To reach the platform, one must walk straight through the barrier between platforms 9 and 10.

In [44]:
reply = chat.predict(human_input="Do you remember my name?")
print(reply)



> Entering new LLMChain chain...
Prompt after formatting:
You are an assistant designed to be able to assist with a wide range of tasks, 
from answering simple questions to providing in-depth explanations and discussions on a wide range of topics.

Use the human input to generate a response that is relevant to the conversation history.
----------
History:

Human: Hi, my name is Adam. I have 3 kids and I like gardening. What is your name?
AI: Hi Adam, it's nice to meet you. I'm Gemini, a large language model from Google AI. I'm glad to hear you enjoy gardening. It's a great way to relax and connect with nature. Do you have any favorite plants or vegetables that you grow?
Human: How long was the last Harry Potter book?
AI: The last Harry Potter book, "Harry Potter and the Deathly Hallows," was published in two volumes. The first volume was released on July 21, 2007, and the second volume was released on July 21, 2007. The first volume is 607 pages long, and the second volume is 652 pag

Yes, I remember your name is Adam. You have 3 kids and you enjoy gardening. You also asked me about the length of 
the last Harry Potter book, the name of their school, and the train platform the train was on. Is there anything 
else I can help you with today?